In [11]:
import Atlas
import random
import pandas as pd
import sys
from tqdm import tqdm

In [12]:
try:
    tape = Atlas.Tape()
except:
    pass

### Market Setup
First, we create a curve. As the time of writing, the only available curve is the ```ZeroRateCurveStrategy```. We create random rates and register them in the tape.

In [13]:
## Create Market Data
evalDate = Atlas.Date(1, Atlas.August, 2020)
store = Atlas.MarketStore(evalDate, Atlas.CLF()) # store with USD as base currency

# create random rates
dt = 0.5
t = []
rates = []
dates = []
startRate = 0.01
for i in range(40):
    rate = Atlas.Dual(startRate + random.random()*0.01)
    tape.registerInput(rate) # we need to register the input to the tape for later use
    rates.append(rate)
    t.append(i*dt)
    dates.append(evalDate + int(i*180))

tape.newRecording() # start recording, for later use

# define curve
curveDayCounter = Atlas.Actual360()
curveCompounding = Atlas.Simple
curveFrequency = Atlas.Annual

strategy = Atlas.ZeroRateLinearStrategy(dates, rates)
curve = Atlas.YieldTermStructure(strategy)
index = Atlas.RateIndex(evalDate, curveFrequency, curveDayCounter, curveFrequency, curveCompounding)
store.addCurve("CLF", curve, index)

### Portfolio Setup

We create a random porfolio of ```FixFloatSwap```, with long and short positions. For simplicity, all trades start today, otherwise we would need to add fixing rates inside the ```RateIndex``` object -a must in a more realistic scenario-.

In [14]:
## set seed
random.seed(1234)
def generateInstruments(numInstruments):
    instruments = []
    for i in tqdm(range(numInstruments)):
        # Generate random start and end dates
        startYear = random.randint(2010, 2020)
        endYear = random.randint(2021, 2035)
        startDate = Atlas.Date(1, Atlas.August, startYear)
        endDate = Atlas.Date(1, Atlas.August, endYear)
        
        # Create the instrument with random parameters
        rateValue = Atlas.Dual(random.uniform(0.01, 0.1))
        dayCounter = Atlas.Thirty360()
        compounding = Atlas.Compounded
        frequency = Atlas.Annual
        rate = Atlas.InterestRate(rateValue, dayCounter, compounding, frequency)

        discountContext = store.curveContext("CLF")
        
        notional = random.uniform(1_000, 10_000)
        paymentFrequency = Atlas.Monthly
        instrument = Atlas.EqualPaymentInstrument(startDate, endDate, paymentFrequency, notional, rate, discountContext)
        instruments.append(instrument)
    return instruments

In [15]:
instruments = generateInstruments(15000)

100%|██████████| 15000/15000 [00:01<00:00, 7664.07it/s]


In [16]:
size = sys.getsizeof(instruments)
print(f"Memory usage: {size/1_000_000} MB")

Memory usage: 0.121432 MB


### Indexing and Pricing

As always, we index the instruments, produce the market data points and price each instrument.

In [17]:
#indexing
indexer = Atlas.Indexer()
for inst in instruments:
    indexer.visit(inst)
request = indexer.request()

#market data simulation
model = Atlas.SpotMarketDataModel(request, store)
marketData = model.marketData(evalDate)

We calculate the NPV of the portfolio.

In [18]:
npv = Atlas.Dual(0.0)
tape.registerOutput(npv)
npvCalculator = Atlas.NPVCalculator(marketData)

for inst in instruments:
    npvCalculator.visit(inst)

npv = npvCalculator.results()
print("NPV: ", npv)

NPV:  Dual(67139273.329363)


### Sensitivities

We calculate the sensitivities using the tape. First we seed and then compute the adjoints. Sensitivies to each rate are recovered by the ```getDerivatives``` method.

In [19]:
npv.setDerivative(0.01)
tape.computeAdjoints()

In [20]:
sens = {}
for dt, rate in zip(t, rates):
    sens[dt] = round(rate.getDerivative(), 4)

df = pd.DataFrame.from_dict(sens, orient='index', columns=['Sensitivity'])
df.index.name = 'Date'
df.T

Date,0.0,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,...,15.0,15.5,16.0,16.5,17.0,17.5,18.0,18.5,19.0,19.5
Sensitivity,-4442.8555,-27272.7473,-48971.6281,-66750.3056,-83505.7128,-94529.921,-107357.5179,-111780.4021,-121005.654,-122044.7226,...,-19563.4884,-2968.1053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
